<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md) 

Lab 4.1 : Spark SQL : Dataframes
================================

### Overview
First look at Spark SQL

### Depends On 
None

### Run time
20-30 mins


----------------------------
STEP 2: Load Clickstream data
----------------------------

In [ ]:
clickstreamDF = spark.read.json("../data/click-stream/clickstream.json")

**==> Monitor Spark shell UI on port 4040**  
You may see something like this:

<img src="../assets/images/5.1a.png" style="border: 5px solid grey; max-width:100%;" />

**==> Q : Why is Spark not lazy loading the JASON files? **

----------------------------
STEP 3 : Inspecting The Dataframe
----------------------------

**==> Print the schema of data frame**     
Hint : `clickstreamDF.printSchema`  
Your output will look like this:

In [ ]:
# TODO: Print the schema



Your output may look like this:


    root
     |-- action: string (nullable = true)
     |-- campaign: string (nullable = true)
     |-- cost: long (nullable = true)
     |-- domain: string (nullable = true)
     |-- ip: string (nullable = true)
     |-- session: string (nullable = true)
     |-- timestamp: long (nullable = true)
     |-- user: string (nullable = true)

**==> Print / Dump the data contained within dataframe**  
Hint : `clickstreamDF.show`

In [ ]:
# TODO: Print/Dump the data



Your output may look like this:

In [ ]:
+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|blocked|campaign_12|   5|     facebook.com|ip_3|session_96|1420070400864|user_5|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
...

   

**==> Explore methods available in Dataframe**  
1) Here is the Dataframe API : 
[Scala](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrame)  /
[Java](http://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrame.html) / 
[Python](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame)


---------------------------------
STEP 4 : Querying Dataframe
---------------------------------

**==> Show only click logs where the cost > 100**

In [ ]:
    clickstreamDF.filter("cost > 100").show()


In [ ]:
    #Same thing
    clickstreamDF.filter(clickstreamDF['cost'] > 100).show()


Sample output

In [ ]:

    +-------+-----------+----+-----------------+----+----------+-------------+------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
    +-------+-----------+----+-----------------+----+----------+-------------+------+
    |clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
    |blocked|campaign_18| 110|    wikipedia.org|ip_5|session_55|1420070402592|user_6|
    |blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|

**==> Show the logs where action = clicked**  
Hint : `clickstreamDF.filter("action == '???'")`  


In [ ]:
# TODO: show the logs where action = clicked


## Step 5 : Do a filter on column type

Use `==`  or `equalTo`

In [11]:

clickstreamDF.filter(clickstreamDF['action'] == 'clicked').show()
clickstreamDF.filter(clickstreamDF['action'].equalTo('clicked')).show()



+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
|clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|
|clicked|campaign_16| 141|    wikipedia.org|ip_1|session_57|1420070409504|user_1|
|clicked| campaign_5|  74|       sfgate.com|ip_6|session_27|1420070412096|user_3|
+-------+-----------+----+-----------------+----+----------+-------------+------+



TypeError: 'Column' object is not callable

## Step 6 : Count the number of visits from each domain

Hint : `g = clickstreamDF.groupBy("column_name")`  
* Inspect methods on `g`  (use tab completion)  
* Then do `count` on `g`  
* Then do a show  
* So `clickstreamDF.groupby().count.show`  


## STEP 7 : Joining Dataframes

Let's load another data set `domain info`  
The data is in   `data/click-stream/domain-info.json`  
The data looks like this:

```
    {"domain":"amazon.com","category":"SHOPPING"}
    {"domain":"bbc.co.uk","category":"NEWS"}
    {"domain":"facebook.com","category":"SOCIAL"}
    ...
```

**==> Load the dataframe**

In [15]:

domainsDF = spark.read.json("../data/click-stream/domain-info.json")

**==> Join both dataframes**

In [19]:

joined = clickstreamDF.join(domainsDF,  clickstreamDF["domain"] == domainsDF["domain"])
    
#see the results
joined.show()

+-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|   category|           domain|
+-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|      VIDEO|      youtube.com|
|blocked|campaign_12|   5|     facebook.com|ip_3|session_96|1420070400864|user_5|     SOCIAL|     facebook.com|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|CLASSIFIEDS|sf.craigslist.org|
|blocked|campaign_18| 110|    wikipedia.org|ip_5|session_55|1420070402592|user_6|  REFERENCE|    wikipedia.org|
|blocked|campaign_17|  20|       amazon.com|ip_4|session_13|1420070406048|user_1|   SHOPPING|       amazon.com|
|blocked|campaign_19| 147|      nytimes.com|ip_2|session_65|1420070407776|user_6|       NEWS|      nytim

**==> Inspect the results, here is a sample**

In [ ]:

    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|   category|           domain|
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    |clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|      VIDEO|      youtube.com|
    |blocked| campaign_2|   7|      youtube.com|ip_2|session_93|1420070412960|user_1|      VIDEO|      youtube.com|
    |blocked|campaign_17|  20|       amazon.com|ip_4|session_13|1420070406048|user_1|   SHOPPING|       amazon.com|

**==> Note some rows are missing.  Which ones?  Why?**

**==> Do an outer join**    
Hint : provide a third argument `outer` to the join statement  
e.g  `val joinedOuter = clickstreamDF.join(domainsDF,  ......,    "outer")`

**==> Inspect the output, might look like this**  

**==> Can you explain the null values?**

In [ ]:
# Complete the TODO:

joinedOuter = clickstreamDF.join(domainsDF, ..... "outer")
joinedOuter.show()

==> Inspect the output, might look like this
==> Can you explain the null values?

```
 joinedOuter.show()
 result ==> 
``` 
```
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|   category|           domain|
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    |blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|       null|             null|
    |   null|       null|null|             null|null|      null|         null|  null|     SOCIAL|      twitter.com|
    |clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|       null|             null|
```